In [62]:
# Importación de librearías
import pandas as pd
import numpy as np

from sklearn import impute
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics

import graphviz

In [63]:
# carga de datos
df = pd.read_csv('../data/raw/train.csv')
df_predict = pd.read_csv('../data/raw/test.csv')

df['tipo'] = "df"
df_predict['tipo'] = "df_predict"
dfw = df.append(df_predict)

In [64]:
# parametros 
p_features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
p_target = "Survived"

In [65]:
# columna Sex
dfw['Sex'][dfw['Sex'] == 'female'] = 0
dfw['Sex'][dfw['Sex'] == 'male'] = 1

# columna Age
imputador_mean = impute.SimpleImputer(missing_values=np.NaN, strategy='mean')
imputador_mean.fit(dfw[['Fare','Age']])
dfw['Age'] = imputador_mean.transform(dfw[['Fare','Age']])[:,1]
dfw['Fare'] = imputador_mean.transform(dfw[['Fare','Age']])[:,0]

# column embarked
dfw['Embarked'][dfw['Embarked'] == 'S'] = 1
dfw['Embarked'][dfw['Embarked'] == 'C'] = 2
dfw['Embarked'][dfw['Embarked'] == 'Q'] = 3

imputador_freq = impute.SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
imputador_freq.fit(dfw[['Sex','Embarked']])
dfw['Embarked'] = imputador_freq.transform(dfw[['Sex','Embarked']])[:,1]

In [66]:
# construcción training testing
X = dfw.loc[dfw['tipo'] == "df", p_features]
y = dfw.loc[dfw['tipo'] == "df", p_target]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 1)

modelo_1 = tree.DecisionTreeClassifier(random_state = 1)
modelo_1 = modelo_1.fit(X_train, y_train)
y_pred_1 = modelo_1.predict(X_test)
acc_modelo_1 = metrics.accuracy_score(y_test, y_pred_1)
metrics.confusion_matrix(y_test, y_pred_1)

array([[129,  24],
       [ 41,  74]])

In [67]:
# Evaluación del modelo: visualización de arbol de decisión
aux = tree.export_graphviz(modelo_1, out_file=None, filled = True, rounded=True, feature_names=p_features)
graf = graphviz.Source(aux)
graf.render('../work/004_decision_tree_basic_2_modelo_a') # exportación a pdf

'../work/004_decision_tree_basic_2_modelo_a.pdf'

In [68]:
# generación modelo kaggle
X_predict = dfw.loc[dfw['tipo'] == "df_predict", p_features]

modelo_final = tree.DecisionTreeClassifier(random_state=1)
modelo_final = modelo_final.fit(X,y)
y_pred_final = modelo_final.predict(X_predict)

df_predict['Survived'] = y_pred_final
df_final = df_predict[['PassengerId', 'Survived']]
df_final['Survived'] = df_final['Survived'].astype(int)
df_final.to_csv('../output/004_decision_tree_basico_a.csv', index=False)

array([0, 1])